# **LoR**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

df = pd.read_csv('healthcare-dataset-stroke-data.csv')

# แยกตัวแปรอิสระ (X) และตัวแปรตาม (y)
X = df.drop(columns=['stroke'])
y = df['stroke']

# แบ่งข้อมูลเป็นชุด train และ test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# ทำการ Normalize ข้อมูล
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# กำหนดค่าพารามิเตอร์สำหรับ Grid Search
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],  # ค่าความแรงของการลงโทษ L2
    'penalty': ['l1', 'l2'],  # ประเภทของ Regularization
    'solver': ['liblinear', 'saga']  # ตัวแก้ปัญหาสำหรับ Logistic Regression
}

# สร้างโมเดล Logistic Regression
log_reg = LogisticRegression(max_iter=1000)

# ใช้ GridSearchCV เพื่อหาค่าพารามิเตอร์ที่ดีที่สุด
grid_search = GridSearchCV(log_reg, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)

# แสดงค่าพารามิเตอร์ที่ดีที่สุด
best_params = grid_search.best_params_
print(best_params)

accuracy_LoR = accuracy_score(y_test, grid_search.predict(X_test_scaled))
print("Accuracy:", accuracy_LoR)

{'C': 0.01, 'penalty': 'l1', 'solver': 'liblinear'}
Accuracy: 0.9510763209393346


#**DT**


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.metrics import accuracy_score

# โหลดข้อมูล
data = pd.read_csv('healthcare-dataset-stroke-data.csv')

# แปลงตัวแปรประเภทเป็นตัวเลขโดยใช้การเข้ารหัสแบบ one-hot
data = pd.get_dummies(data, drop_first=True)

# แยกคุณสมบัติและตัวแปรเป้าหมาย
X = data.drop(columns=["stroke"])  # คุณสมบัติ
y = data["stroke"]  # เป้าหมาย

# แบ่งข้อมูลเป็นชุดฝึกและชุดทดสอบ
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# เลือกคุณสมบัติ K อันดับแรกโดยใช้ข้อมูลร่วม
k = 5  # จำนวนคุณสมบัติที่ต้องการเลือก
selector = SelectKBest(score_func=mutual_info_classif, k=k)
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)

# ดึงชื่อคุณสมบัติที่ถูกเลือก
selected_features = X_train.columns[selector.get_support()]
print("Selected Features:", selected_features)

# กำหนดตารางพารามิเตอร์สำหรับ GridSearchCV
param_grid = {
    'criterion': ['gini', 'entropy'],  # เกณฑ์การแยก: gini หรือ entropy
    'max_depth': [3, 5, 7, 10, None],  # ความลึกสูงสุด
    'min_samples_split': [2, 5, 10],   # จำนวนตัวอย่างขั้นต่ำสำหรับการแยก
    'min_samples_leaf': [1, 2, 4],     # จำนวนตัวอย่างขั้นต่ำที่ใบ
    'max_features': [None, 'sqrt', 'log2']  # จำนวนคุณสมบัติสูงสุดที่พิจารณา
}

# สร้างตัวจำแนกแบบต้นไม้ตัดสินใจ
DT_model = DecisionTreeClassifier(random_state=42)

# ใช้ GridSearchCV
grid_search = GridSearchCV(
    estimator=DT_model,
    param_grid=param_grid,
    cv=5,  # การตรวจสอบข้าม 5 ส่วน
    scoring='accuracy',  # วัดด้วยความแม่นยำ
    n_jobs=-1,  # ใช้ทุกคอร์ของ CPU
    verbose=1  # แสดงความคืบหน้า
)

# ฝึกโมเดลด้วยการค้นหาแบบกริด
grid_search.fit(X_train_selected, y_train)

# แสดงพารามิเตอร์และคะแนนที่ดีที่สุด
print("\nBest Parameters:", grid_search.best_params_)
print("Best Cross-validation Score:", grid_search.best_score_)

# ทำนายผลด้วยโมเดลที่ดีที่สุด
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_selected)

# แสดงผลลัพธ์สุดท้าย
accuracy_DT = accuracy_score(y_test, y_pred)
print("Test Set Accuracy:", accuracy_DT)


Selected Features: Index(['age', 'hypertension', 'heart_disease', 'ever_married', 'bmi'], dtype='object')
Fitting 5 folds for each of 270 candidates, totalling 1350 fits

Best Parameters: {'criterion': 'gini', 'max_depth': 3, 'max_features': None, 'min_samples_leaf': 4, 'min_samples_split': 2}
Best Cross-validation Score: 0.9542565830622498
Test Set Accuracy: 0.9393346379647749


#**KNN**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report

# โหลดข้อมูล
data = pd.read_csv('healthcare-dataset-stroke-data.csv')

# แปลงตัวแปรเชิงพาณิชย์ให้เป็นตัวเลขด้วย One-Hot Encoding
data = pd.get_dummies(data, drop_first=True)

# แยกคุณลักษณะ (features) และตัวแปรเป้าหมาย (target)
X = data.drop(columns=["stroke"])
y = data["stroke"]

# แบ่งข้อมูลเป็นชุดฝึก (train) และชุดทดสอบ (test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ปรับขนาดคุณลักษณะ (Feature Scaling)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# กำหนด parameters สำหรับ Grid Search
param_grid = {
    'n_neighbors': list(range(1, 31)),  # ทดสอบ k จาก 1 ถึง 30
    'weights': ['uniform', 'distance'],  # ลองทั้งแบบ uniform และ distance
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],  # อัลกอริทึมในการค้นหา
    'p': [1, 2]  # Minkowski metric กับ p=1 (Manhattan) และ p=2 (Euclidean)
}

# สร้าง GridSearchCV
knn_grid = GridSearchCV(
    estimator=KNeighborsClassifier(),
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    scoring='accuracy',  # วัดประสิทธิภาพด้วย accuracy
    n_jobs=-1  # ใช้ CPU ทั้งหมด
)

# ฝึกโมเดลด้วย Grid Search
knn_grid.fit(X_train_scaled, y_train)

# พิมพ์ผลลัพธ์ที่ดีที่สุด
print("Best Parameters:", knn_grid.best_params_)
print("Best Cross-validation Score:", knn_grid.best_score_)

# ทำนายผลลัพธ์ด้วยโมเดลที่ดีที่สุด
best_knn = knn_grid.best_estimator_
y_pred = best_knn.predict(X_test_scaled)

# ประเมินประสิทธิภาพโมเดล
print("\nTest Set Performance:")
print("Accuracy:", accuracy_score(y_test, y_pred))


Best Parameters: {'algorithm': 'auto', 'n_neighbors': 6, 'p': 2, 'weights': 'uniform'}
Best Cross-validation Score: 0.9542565830622498

Test Set Performance:
Accuracy: 0.9393346379647749


# **GB**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler

# โหลดข้อมูล
data = pd.read_csv('healthcare-dataset-stroke-data.csv')

# แปลงตัวแปรเชิงพาณิชย์ให้เป็นตัวเลขด้วย One-Hot Encoding
data = pd.get_dummies(data, drop_first=True)  # drop_first=True เพื่อหลีกเลี่ยงปัญหาของ dummy variable trap

# แยกคุณลักษณะ (features) และตัวแปรเป้าหมาย (target)
X = data.drop(columns=["stroke"])  # ใช้ทุกคอลัมน์ยกเว้นตัวแปรเป้าหมาย
y = data["stroke"]  # ตัวแปรเป้าหมาย

# แบ่งข้อมูลเป็นชุดฝึก (train) และชุดทดสอบ (test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ปรับขนาดคุณลักษณะ (Feature Scaling)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# กำหนดตารางพารามิเตอร์สำหรับ RandomizedSearchCV
param_dist = {
    'n_estimators': [100, 200],  # ลดจำนวนต้นไม้เพื่อลดเวลา
    'learning_rate': [0.01, 0.1],  # ลดจำนวนค่าที่ต้องค้นหา
    'max_depth': [3, 5],  # ลดจำนวนค่าความลึก
    'min_samples_split': [2, 5],  # ลดค่า split ที่ต้องลอง
    'min_samples_leaf': [1, 2]  # ลดค่า leaf ที่ต้องลอง
}

# สร้างโมเดล Gradient Boosting
gb_model = GradientBoostingClassifier(random_state=42)

# ใช้ RandomizedSearchCV แทน GridSearchCV
random_search = RandomizedSearchCV(
    estimator=gb_model,
    param_distributions=param_dist,
    n_iter=20,  # ลดจำนวนชุดค่าพารามิเตอร์ที่ต้องลอง
    cv=3,  # ลดจำนวน cross-validation folds
    scoring='accuracy',  # วัดด้วยความแม่นยำ
    n_jobs=-1,  # ใช้ทุกคอร์ของ CPU
    random_state=42
)

# ฝึกโมเดลด้วยการค้นหาแบบสุ่ม
random_search.fit(X_train, y_train)

# แสดงพารามิเตอร์และคะแนนที่ดีที่สุด
print("\nBest Parameters:", random_search.best_params_)
print("Best Cross-validation Score:", random_search.best_score_)

# ทำนายผลด้วยโมเดลที่ดีที่สุด
best_model = random_search.best_estimator_
y_pred_gb = best_model.predict(X_test)

# แสดงผลลัพธ์สุดท้าย
accuracy_gb = accuracy_score(y_test, y_pred_gb)
print("Gradient Boosting Test Set Accuracy:", accuracy_gb)


Best Parameters: {'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_depth': 3, 'learning_rate': 0.01}
Best Cross-validation Score: 0.954256414455315
Gradient Boosting Test Set Accuracy: 0.9393346379647749


# **ADABOOST**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import StandardScaler

# โหลดข้อมูล
data = pd.read_csv('healthcare-dataset-stroke-data.csv')

# แปลงตัวแปรเชิงพาณิชย์ให้เป็นตัวเลขด้วย One-Hot Encoding
data = pd.get_dummies(data, drop_first=True)  # drop_first=True เพื่อหลีกเลี่ยงปัญหาของ dummy variable trap

# แยกคุณลักษณะ (features) และตัวแปรเป้าหมาย (target)
X = data.drop(columns=["stroke"])  # ใช้ทุกคอลัมน์ยกเว้นตัวแปรเป้าหมาย
y = data["stroke"]  # ตัวแปรเป้าหมาย

# แบ่งข้อมูลเป็นชุดฝึก (train) และชุดทดสอบ (test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ปรับขนาดคุณลักษณะ (Feature Scaling)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# กำหนดตารางพารามิเตอร์สำหรับ RandomizedSearchCV
param_dist = {
    'n_estimators': [50, 100],  # ลดจำนวนตัวประมาณการเพื่อลดเวลา
    'learning_rate': [0.01, 0.1]  # ลดจำนวนค่าที่ต้องค้นหา
}

# สร้างโมเดล AdaBoost
ada_model = AdaBoostClassifier(random_state=42)

# ใช้ RandomizedSearchCV แทน GridSearchCV
random_search = RandomizedSearchCV(
    estimator=ada_model,
    param_distributions=param_dist,
    n_iter=10,  # ลดจำนวนชุดค่าพารามิเตอร์ที่ต้องลอง
    cv=3,  # ลดจำนวน cross-validation folds
    scoring='accuracy',  # วัดด้วยความแม่นยำ
    n_jobs=-1,  # ใช้ทุกคอร์ของ CPU
    random_state=42
)

# ฝึกโมเดลด้วยการค้นหาแบบสุ่ม
random_search.fit(X_train, y_train)

# แสดงพารามิเตอร์และคะแนนที่ดีที่สุด
print("\nBest Parameters:", random_search.best_params_)
print("Best Cross-validation Score:", random_search.best_score_)

# ทำนายผลด้วยโมเดลที่ดีที่สุด
best_model = random_search.best_estimator_
y_pred_ada = best_model.predict(X_test)

# แสดงผลลัพธ์สุดท้าย
accuracy_ada = accuracy_score(y_test, y_pred_ada)
print("AdaBoost Test Set Accuracy:", accuracy_ada)

/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_search.py:317: UserWarning: The total space of parameters 4 is smaller than n_iter=10. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(



Best Parameters: {'n_estimators': 50, 'learning_rate': 0.01}
Best Cross-validation Score: 0.954256414455315
AdaBoost Test Set Accuracy: 0.9393346379647749


# **RANDOM FOREST**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

# โหลดข้อมูล
data = pd.read_csv('healthcare-dataset-stroke-data.csv')

# แปลงตัวแปรเชิงพาณิชย์ให้เป็นตัวเลขด้วย One-Hot Encoding
data = pd.get_dummies(data, drop_first=True)

# แยกคุณลักษณะ (features) และตัวแปรเป้าหมาย (target)
X = data.drop(columns=["stroke"])
y = data["stroke"]

# แบ่งข้อมูลเป็นชุดฝึก (train) และชุดทดสอบ (test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ปรับขนาดข้อมูล (Standardization)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# กำหนดช่วงค่าพารามิเตอร์สำหรับ Randomized Search
param_dist = {
    'n_estimators': np.arange(50, 301, 50),
    'max_depth': np.arange(3, 21, 3),
    'min_samples_split': np.arange(2, 11, 2),
    'min_samples_leaf': np.arange(1, 11, 2),
    'max_features': ['sqrt', 'log2']
}

# สร้างโมเดล Random Forest
rf_model = RandomForestClassifier(random_state=42)

# ใช้ Randomized Search
random_search = RandomizedSearchCV(
    rf_model,
    param_distributions=param_dist,
    n_iter=20,  # ลองพารามิเตอร์ 20 ชุด
    cv=5,       # ใช้ 5-fold cross-validation
    scoring='accuracy',
    n_jobs=-1,
    random_state=42
)

# ฝึกโมเดล (หลังจากที่แน่ใจว่า X_train มีค่าจริง)
random_search.fit(X_train, y_train)

# แสดงค่าพารามิเตอร์ที่ดีที่สุด
print("Best Parameters:", random_search.best_params_)
print("Best Accuracy:", random_search.best_score_)

# ใช้โมเดลที่ดีที่สุด
best_rf_model = random_search.best_estimator_

Best Parameters: {'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 7, 'max_features': 'sqrt', 'max_depth': 18}
Best Accuracy: 0.9542565830622498


# **VOTING**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, RandomForestClassifier, VotingClassifier
from sklearn.metrics import accuracy_score

# โหลดข้อมูล
data = pd.read_csv('healthcare-dataset-stroke-data.csv')

# แปลงตัวแปรหมวดหมู่เป็นตัวเลขด้วย One-Hot Encoding
data = pd.get_dummies(data, drop_first=True)

# แยกตัวแปรอิสระและตัวแปรตาม
X = data.drop(columns=["stroke"])
y = data["stroke"]

# แบ่งข้อมูลเป็น train และ test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# ปรับขนาดข้อมูล (Standardization)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# โหลดโมเดลที่ดีที่สุดจาก Randomized Search ของแต่ละตัว
log_reg = LogisticRegression(C=1, penalty='l2', solver='liblinear', max_iter=1000)
dt = DecisionTreeClassifier(criterion='gini', max_depth=5, min_samples_split=5, min_samples_leaf=2, max_features='sqrt', random_state=42)
knn = KNeighborsClassifier(n_neighbors=5, weights='distance', algorithm='auto', p=2)
gb = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, min_samples_split=5, min_samples_leaf=2, random_state=42)
ada = AdaBoostClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
rf = RandomForestClassifier(n_estimators=200, max_depth=10, min_samples_split=5, min_samples_leaf=2, max_features='sqrt', random_state=42)

# สร้าง Voting Classifier
voting_clf = VotingClassifier(estimators=[
    ('Logistic Regression', log_reg),
    ('Decision Tree', dt),
    ('KNN', knn),
    ('Gradient Boosting', gb),
    ('AdaBoost', ada),
    ('Random Forest', rf)
], voting='soft')

# กำหนดพารามิเตอร์สำหรับ Randomized Search
param_dist = {
    'weights': [[1, 1, 1, 1, 1, 1], [2, 1, 1, 1, 1, 1], [1, 2, 1, 1, 1, 1], [1, 1, 2, 1, 1, 1], [1, 1, 1, 2, 1, 1]]
}

# ใช้ Randomized Search เพื่อหา Voting Weight ที่ดีที่สุด
random_search = RandomizedSearchCV(voting_clf, param_distributions=param_dist, n_iter=5, cv=3, scoring='accuracy', n_jobs=-1, random_state=42)
random_search.fit(X_train_scaled, y_train)

# แสดงค่าพารามิเตอร์ที่ดีที่สุด
print("Best Parameters:", random_search.best_params_)
print("Best Cross-validation Score:", random_search.best_score_)

# ทำนายผลลัพธ์ด้วยโมเดลที่ดีที่สุด
y_pred = random_search.best_estimator_.predict(X_test_scaled)

# แสดงผลลัพธ์สุดท้าย
accuracy_voting = accuracy_score(y_test, y_pred)
print("Voting Classifier Test Set Accuracy:", accuracy_voting)

Best Parameters: {'weights': [1, 1, 1, 1, 1, 1]}
Best Cross-validation Score: 0.9513209933602886
Voting Classifier Test Set Accuracy: 0.9510763209393346
